In [21]:
import numpy as np
import annfore

In [22]:
from annfore.net import nn_sir_path_obs
from annfore.models import sir_model_N_obs
from annfore.utils.graph import find_neighs

from annfore.learn.opt import make_opt
from annfore.learn.losses import loss_fn_coeff
from annfore.learn.train import train_beta, make_training_step_local

In [23]:
device="cpu"

In [24]:
# contact array of with entries [time, node_i, node_j, lambda]
# ordered by time
contacts = np.array([
    [0,1,0, 0.5],
    [0,0,1, 0.5],
    [0,2,0, 0.5],
    [0,0,2, 0.5],
    [2,3,0, 0.5],
    [2,0,3, 0.5],
    [3,1,0, 0.5],
    [3,0,1, 0.5],
])

# observations [node, state, time] -- state 0,1,2 for S,I,R
# ordered by time

obs = [
    [1,1,0],
    [2,0,0],
    [3,1,3],
    ]

N = int(max(contacts[:, 1]) + 1)
t_limit = int(max(contacts[:, 0]) + 1) # t_limit times || +1 obs after contacts
mu=0.1


In [25]:
# SIR model
model = sir_model_N_obs.SirModel(contacts, 
                                mu = mu,
                               device = device)
model.set_obs(obs)

In [26]:
# define the autoregressive neurla network
dependece_net = find_neighs(contacts,N=N,only_minor=True, next_near_neigh=True)
net = nn_sir_path_obs.SIRPathColdObs(dependece_net,
                    t_limit+1, # +1 for susceptible
                    obs_list=obs,
                    hidden_layer_spec=[1,1],
                    )

In [27]:
# define the optimizer over the parameters of the net
optimizer = []
lr = 1e-3
for i in range(N):
    if len(net.params_i[i]):
        optimizer.append(make_opt(net.params_i[i], lr=lr))


In [30]:
t_obs = 0
betas = np.arange(0,1, 1e-3)
num_samples = 100
results = train_beta(net, optimizer,
                        model, "out.txt",
                        loss_fn_coeff, t_obs,
                        num_samples=num_samples,
                        train_step = make_training_step_local,
                        betas=betas, save_every=200)

 200 beta: 0.2000, loss: 100.355, std: 18.045, ener: 109.13, max_grad = 0.0, count_zero_pw = 141.00, num_I = 1.620, num_R = 0.1000 (T_obs=0) -- took      7 ms: sample: 1, log_prob: 0, trans_sample: 0, energy: 1, loss: 0, optim_step: 3, stats: 0 -- source: {   1:1.000,   3:0.440,   0:0.180,   2:0.000}    - remain time:  5 s, 646 msdict_keys(['step', 'beta', 'energy', 'std_energy', 'loss', 'loss_std', 'S', 'I', 'R', 't_obs', 'num_source', 'sources', 'max_grad', 'num_zero_pw', 'N', 'T', 'p_source', 'p_sus', 'p_obs', 'p_w', 'mu', 'times'])
 400 beta: 0.4000, loss: 72.050, std: 10.798, ener: 76.83, max_grad = 0.0, count_zero_pw = 22.00, num_I = 1.370, num_R = 0.0100 (T_obs=0) -- took      7 ms: sample: 1, log_prob: 0, trans_sample: 0, energy: 1, loss: 0, optim_step: 3, stats: 0 -- source: {   1:1.000,   3:0.200,   0:0.170,   2:0.000}    - remain time:  4 s, 232 msdict_keys(['step', 'beta', 'energy', 'std_energy', 'loss', 'loss_std', 'S', 'I', 'R', 't_obs', 'num_source', 'sources', 'max_grad

In [32]:
M = net.marginals()

In [37]:
M[:, 0,1] # marginal probability to be infected of nodes (0,1,2,3) at time t = 0

tensor([0.0071, 1.0000, 0.0000, 0.0056])